### Requirements

In [ ]:
!pip install findspark
!pip install pyspark

In [ ]:
!pip install request 

In [ ]:
import pandas as pd
import bz2
import json
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf
import requests
import findspark

In [ ]:
findspark.init('/Users/tatianacogne/spark')

### Objectives M2
- That you can handle the data in its size.
- That you understand what’s in the data (formats, distributions, missing values, correlations, etc.).
- That you considered ways to enrich, filter, transform the data according to your needs.
- That you have a reasonable plan and ideas for methods you’re going to use, giving their essential mathematical details in the notebook.
- That your plan for analysis and communication is reasonable and sound, potentially discussing alternatives to your choices that you considered but dropped.

### Test with PySpark
https://spark.apache.org/docs/latest/sql-programming-guide.html

In [ ]:
# Create a spark context
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

# Read JSON file into dataframe
df = spark.read.json('data/quotes-2020.json.bz2')

In [ ]:
findspark.init() 

### Summary Columns
- **quoteID**:      Primary key of the quotation (format: "YYYY-MM-DD-{increasing int:06d}")
- **quotation**:    Text of the longest encountered original form of the quotation
- **speaker**:      Selected most likely speaker
- **qids**:         Wikidata IDs of all aliases that match the selected speaker
- **date**:         Earliest occurrence date of any version of the quotation
- **numOccurences**:Number of time this quotation occurs in the articles
- **probas**:       Array representing the probabilities of each speaker having uttered the quotation
- **urls**:         List of links to the original articles containing the quotation
- **phase**:        Corresponding phase of the data in which the quotation first occurred (A-E)
- **domains**:      Domain of the URL 

In [ ]:
df.printSchema()

In [ ]:
df.show()

# Tasks

## Analysis 

- quids same for each quote 
- check proba avec le speaker 
- check chaque colonne
- verifier l'URL 

In [ ]:
# speaker
df_speakers =df.drop_duplicates(subset=['speaker'])

In [ ]:
num_diff_speakers = df_speakers.count()

In [ ]:
df_names = df_speakers[['speaker']]

In [ ]:
df_none = df[df.speaker=='None']

In [ ]:
#CHECK: speaker == None don't have qids --> OK 
df_none.filter(sf.size('qids') > 0).show()

In [ ]:
def valid_urls(urls):
    bool_urls = []
    for u in urls:
        bool_urls.append(requests.get(u).ok)
    return any(bool_urls)

In [ ]:
#CHECK: valid URL (not working)
df.select(filter("urls", valid_urls))

In [ ]:
a= sf.split(df['spearker'], ' ')
#df[df.speaker.str.contains('Trump')]

In [ ]:
url = 'https://stackoverflow.com/questions/54087303/python-requests-how-to-check-for-200-ok'
requests.get(url).ok

#### Analysis on speakers 
- Number of different speakers : 218415

# Draft 

#### Test with DataFrame (DON'T RUN THIS CELL)

In [ ]:
%%time
#df_quotes_2020 = pd.read_json('data/quotes-2020.json.bz2', compression='bz2',lines=True)